the task is simple. scrape some more data from wunderground. pipeline:
- get time value of the date today in a variable, datetime it, then get tomorrow's values assigned to a variable. pandas should be able to handle it. then i do need to get tomorrow's values in a format such as yyyy-mm-dd for the site. example url: https://www.wunderground.com/hourly/be/leuven/date/2023-05-06 for hourly predictions of 06 may 2023.
- use the previous scraping pipeline off the scrape_wunderground_daily.ipynb file. the modifications will have to be made in terms of the conversion of the scraped data into a proper dataframe, while the capturing of the data with bs4 and selenium itself remains the same.
- once the dataframe is converted properly, preferably export with the format 'WG_hr_20230506.csv' for instance. we can just modify this if situations change.

unfortunate nature is that selenium will be required to actually render wunderground's contents as we see it, then render it. only requesting the soup will cause in the table not actually being there.

for reference, this is what a row looks like: \
'12:00 am',
 'Mostly Clear',
 '56\xa0°F',
 '56\xa0°F',
 '6\xa0°%',
 '0\xa0°in',
 '22\xa0°%',
 '51\xa0°F',
 '83\xa0°%',
 '4\xa0°mph SSW',
 '30.02\xa0°in',

there are 11 measurements in a row. therefore the first obs will be data[0] to data[10], then second obs data[11] to data[21] \
the target scrape can have different row numbers, so it is best to get the code to work off the length of the scrape results. 

i make a bigass pipe function for converting the scraped data into a dataframe, defining: \
colnames = ['Time','Conditions','Temperature','Feels_Like','Precipitation_Chance','Daily_Rain_Sum','Cloud_Cover','Dew_Point','Humidity','Wind','Pressure']
for the function:
hour_mess(date_iter,data,colnames=colnames) \

In [118]:
#here's an example of how it does for one particular day.

,Time,Conditions,Temperature,Feels_Like,Precipitation_Chance,Daily_Rain_Sum,Cloud_Cover,Dew_Point,Humidity,Wind,Pressure
0,2023-05-06 00:00:00+00:00,Mostly Clear,13.333333,13.333333,6,0.0,22,10.555556,83,1.788109,1015.883721
1,2023-05-06 01:00:00+00:00,Mostly Clear,12.222222,11.666667,6,0.0,29,10.555556,90,1.788109,1015.883721
2,2023-05-06 02:00:00+00:00,Partly Cloudy,11.666667,11.111111,3,0.0,30,10.000000,91,1.788109,1015.883721
3,2023-05-06 03:00:00+00:00,Partly Cloudy,11.111111,10.555556,3,0.0,41,10.000000,93,1.788109,1015.883721
4,2023-05-06 04:00:00+00:00,Mostly Cloudy,10.555556,10.555556,8,0.0,69,10.000000,94,1.788109,1015.883721
5,2023-05-06 05:00:00+00:00,Cloudy,10.000000,10.000000,16,0.0,81,9.444444,95,1.341082,1015.883721
6,2023-05-06 06:00:00+00:00,Cloudy,10.555556,10.000000,16,0.0,85,9.444444,94,1.341082,1015.883721
7,2023-05-06 07:00:00+00:00,Cloudy,10.555556,10.555556,12,0.0,83,10.000000,95,1.341082,1015.883721
8,2023-05-06 08:00:00+00:00,Cloudy,12.222222,12.222222,11,0.0,80,10.555556,90,1.341082,1015.883721
9,2023-05-06 09:00:00+00:00,Cloudy,13.888889,13.888889,12,0.0,85,11.666667,85,1.341082,1015.883721


now that the kinks have been sorted, we gather all that code here in one massive procedure:

In [130]:
#source: https://bojanstavrikj.github.io/content/page1/wunderground_scraper
import requests #library needed for requesting permission to website
import pandas as pd
import selenium
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from functools import reduce
import time
import chromedriver_binary
import datetime

colnames = ['Time','Conditions','Temperature','Feels_Like','Precipitation_Chance',
            'Daily_Rain_Sum','Cloud_Cover','Dew_Point','Humidity','Wind','Pressure']
def hour_mess(date_iter,data,colnames=colnames):
    timevals = (pd.DataFrame([data[i:i+10] for i in range(0, len(data), 11 )])
            .pipe(lambda x: x.iloc[:, 0]).rename('datetime',inplace=True).to_frame()
            .pipe(lambda x: x.assign(Time=date_iter + ' ' + x['datetime'])).drop('datetime',axis=1)
            .pipe(lambda x: x.assign(Time=pd.to_datetime(x['Time'], format='%Y-%m-%d %I:%M %p',utc=True))))
    cvals = pd.DataFrame([data[i] for i in range(1, len(data), 11)], columns=[colnames[1]])
    tempvals = (pd.DataFrame([data[i:i+10] for i in range(2, len(data), 11)])
           .pipe(lambda x: x.iloc[:, 0]).pipe(lambda x: x.str.extract('(\d+)')).pipe(lambda x: x.astype(float))
           .pipe(lambda x: x.rename(columns={0: 'Temperature'})).pipe(lambda x: (x-32)*5/9))
    flvals = (pd.DataFrame([data[i:i+10] for i in range(3, len(data), 11)])
           .pipe(lambda x: x.iloc[:, 0]).pipe(lambda x: x.str.extract('(\d+)')).pipe(lambda x: x.astype(float))
           .pipe(lambda x: x.rename(columns={0: 'Feels_Like'})).pipe(lambda x: (x-32)*5/9))
    pcvals = (pd.DataFrame([data[i:i+10] for i in range(4, len(data), 11)])
           .pipe(lambda x: x.iloc[:, 0]).pipe(lambda x: x.str.extract('(\d+)'))
           .pipe(lambda x: x.astype(int)).pipe(lambda x: x.rename(columns={0: 'Precipitation_Chance'})))
    drsvals = (pd.DataFrame([data[i:i+10] for i in range(5, len(data), 11)])
           .pipe(lambda x: x.iloc[:, 0]).pipe(lambda x: x.str.extract('(\d+)')).pipe(lambda x: x.astype(int))
           .pipe(lambda x: x.rename(columns={0: 'Daily_Rain_Sum'})).pipe(lambda x: x*25.4))
    ccvals = (pd.DataFrame([data[i:i+10] for i in range(6, len(data), 11)])
           .pipe(lambda x: x.iloc[:, 0]).pipe(lambda x: x.str.extract('(\d+)'))
           .pipe(lambda x: x.astype(int)).pipe(lambda x: x.rename(columns={0: 'Cloud_Cover'})))
    dpvals = (pd.DataFrame([data[i:i+10] for i in range(7, len(data), 11)])
           .pipe(lambda x: x.iloc[:, 0]).pipe(lambda x: x.str.extract('(\d+)'))
           .pipe(lambda x: x.astype(float)).pipe(lambda x: x.rename(columns={0: 'Dew_Point'})).pipe(lambda x: (x-32)*5/9))
    hvals = (pd.DataFrame([data[i:i+10] for i in range(8, len(data), 11)])
           .pipe(lambda x: x.iloc[:, 0]).pipe(lambda x: x.str.extract('(\d+)'))
           .pipe(lambda x: x.astype(int)).pipe(lambda x: x.rename(columns={0: 'Humidity'})))
    wvals = (pd.DataFrame([data[i:i+10] for i in range(9, len(data), 11)])
           .pipe(lambda x: x.iloc[:, 0]).pipe(lambda x: x.str.extract('(\d+)')).pipe(lambda x: x.astype(float)).pipe(lambda x: x / 2.237)
           .pipe(lambda x: x.rename(columns={0: 'Wind'})))
    pvals = (pd.DataFrame([data[i:i+9] for i in range(10, len(data), 11)])
           .pipe(lambda x: x.iloc[:, 0]).pipe(lambda x: x.str.extract('(\d+)')).pipe(lambda x: x.astype(int))
           .pipe(lambda x: x.rename(columns={0: 'Pressure'})).pipe(lambda x: x*33.8627906977))

    df_data = pd.concat(objs=[timevals,cvals,tempvals,flvals,pcvals,
                              drsvals,ccvals,dpvals,hvals,wvals,pvals],axis=1)
    return df_data

def render_page(url):
        driver = webdriver.Chrome()
        driver.get(url)
        driver.implicitly_wait(5)
        time.sleep(5) #open content and so it fully renders, then parse it. crawl delay is 10 seconds at least, so make it 11 for ethics.
        r = driver.page_source
        driver.quit()
        return r

dates = [datetime.datetime.now().strftime('%Y-%m-%d')] #assume only one day is needed for this one.
page = 'https://www.wunderground.com/hourly/be/leuven/date/' #the main page for the leuven hourly forecast data.

combined_output = pd.DataFrame()
#colnames has been defined above in the collapsed cell block
for date_iter in dates:
    url = str(str(page)+str(date_iter))
    r = render_page(url)
    soup = BS(r, "html.parser")
    container = soup.find('lib-city-hourly-forecast')
    check = container.find('tbody') #the part of the site i actually want

    data = []
    for c in check.find_all('tr', class_='mat-row cdk-row ng-star-inserted'):
        for i in c.find_all('td', class_='ng-star-inserted'):
            trial = i.text
            trial = trial.strip('  ')
            data.append(trial)
    
    output=hour_mess(date_iter=date_iter,data=data,colnames=colnames)
    combined_output = pd.concat(objs=[combined_output,output])
combined_output.to_csv(path_or_buf='RTweather_'+dates[0].replace('-','')+'.csv',sep=';',index=False)


the only real weakness i can see to this scraping is that for times such as 23u and over, 

wunderground will instead show the predictions for the next day while the date used in the finished dataframe is that of the day the above function is called. will have to discuss whether this is a potential problem. 

in the meantime this is done.